### 数据展示

In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from sklearn import linear_model
import xgboost as xgb
# from ultis import *

道路通行时间：

In [6]:
df = pd.read_csv('new_gy_contest_traveltime_training_data_second.txt',delimiter=';',dtype={'link_ID':object})
df.head()

,link_ID,date,time_interval,travel_time
0,4377906283422600514,2017-05-06,"[2017-05-06 11:04:00,2017-05-06 11:06:00)",3.0
1,3377906289434510514,2017-05-06,"[2017-05-06 10:42:00,2017-05-06 10:44:00)",1.0
2,3377906285934510514,2017-05-06,"[2017-05-06 11:56:00,2017-05-06 11:58:00)",35.2
3,3377906285934510514,2017-05-06,"[2017-05-06 17:46:00,2017-05-06 17:48:00)",26.2
4,3377906287934510514,2017-05-06,"[2017-05-06 10:52:00,2017-05-06 10:54:00)",10.4


time_interval时间间隔，两分钟为单位

travel_time平均通行时间

道理长宽情况：

In [8]:
link_df = pd.read_csv('gy_contest_link_info.txt',delimiter=';',dtype={'link_ID':object})
link_df.head()

,link_ID,length,width,link_class
0,4377906289869500514,57,3,1
1,4377906284594800514,247,9,1
2,4377906289425800514,194,3,1
3,4377906284525800514,839,3,1
4,4377906284422600514,55,12,1


length长度    width宽度    link_class类别

道路之间连接情况：

In [11]:
link_tops = pd.read_csv('gy_contest_link_top_update.txt',delimiter=',',dtype={'link_ID':object})
link_tops.head()

,link_ID,in_links,out_links
0,4377906289869500514,1,1
1,4377906284594800514,1,1
2,4377906289425800514,1,1
3,4377906284525800514,1,1
4,4377906284422600514,2,1


### 任务：预测未来一个月平均通行结果，每两分钟一次
回归任务

构建时间序列，基于前几天或者前几十天的数据预测